<a id='top'></a>

# FBref Player Stats Web Scraping
##### Notebook to scrape raw data from [FBref](https://fbref.com/en/) via [StatsBomb](https://statsbomb.com/) using the [pandas](http://pandas.pydata.org/)

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 31/08/2020<br>
Notebook last updated: 31/08/2021

![title](../../img/fbref-logo-banner.png)

![title](../../img/stats-bomb-logo.png)

___

<a id='sectionintro'></a>

## <a id='import_libraries'>Introduction</a>
This notebook scrapes player statstics data from [FBref](https://fbref.com/en/) via [StatsBomb](https://statsbomb.com/), using the [pandas](http://pandas.pydata.org/) [`read_html`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) function [pandas](http://pandas.pydata.org/) for webscraping and data manipulation through DataFrames.

For more information about this notebook and the author, I am available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/A%29%20Web%20Scraping/FBref%20Web%20Scraping%20and%20Parsing.ipynb).

___

<a id='sectioncontents'></a>

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Scraping](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Outfielder Players](#section3.2)<br>
            1.    [Data Dictionary](#section3.2.1)<br>
            2.    [Creating the DataFrame](#section3.2.2)<br>
            3.    [Initial Data Handling](#section3.2.3)<br>
            4.    [Export the Raw DataFrame](#section3.2.4)<br>
      3.    [Goalkeepers](#section3.3)<br>
            1.    [Data Dictionary](#section3.3.1)<br>
            2.    [Creating the DataFrame](#section3.3.2)<br>
            3.    [Initial Data Handling](#section3.3.3)<br>
            4.    [Export the Raw DataFrame](#section3.3.4)<br> 
4.    [Summary](#section4)<br>
5.    [Next Steps](#section5)<br>
6.    [References](#section6)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>

This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing; and
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation.

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [12]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
import pandas_profiling as pp
import os
import re
import random
import glob
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os

# Working with JSON
import json
from pandas.io.json import json_normalize

# Web Scraping
import requests
from bs4 import BeautifulSoup
import re

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [13]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))

Python: 3.7.6
NumPy: 1.20.3
pandas: 1.3.2
matplotlib: 3.4.2


### Define Filepaths

In [14]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..')
data_dir = os.path.join(base_dir, 'data')
data_dir_fbref = os.path.join(base_dir, 'data', 'fbref')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Defined Variables

In [15]:
# Defined variables

## Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Dictionaries

In [16]:
# Define league names and their IDs
dict_league_names = {'Premier-League': '9',
                     'Ligue-1': '13',
                     'Bundesliga': '20',
                     'Serie-A': '11',
                     'La-Liga': '12',
                     'Major-League-Soccer': '22',
                     'Big-5-European-Leagues': 'Big5'
                    }

### Defined Lists

In [17]:
# Defined Lists

## Define list of long names for 'Big 5' European Leagues and MLS
lst_league_names_long = ['Premier-League', 'Ligue-1', 'Bundesliga', 'Serie-A', 'La-Liga', 'Major-League-Soccer', 'Big-5-European-Leagues']

## Define seasons to scrape
lst_seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

## Define list of folders
lst_folders = ['raw', 'engineered', 'reference']

## Define list of data types
lst_data_types = ['goalkeeper', 'outfield', 'team']

### Custom Functions (Scrapers)

In [18]:
# Define function for scraping a defined season and competition of FBref player data
def get_fbref_player_stats(lst_league_names, lst_seasons):
    
    """
    Function to scrape player stats from FBref.
    """
    
    
    ## Define list of league names
    league_names_long = lst_league_names
    
    
    ## Define seasons to scrape
    seasons = lst_seasons
    
    
    ## Start timer
    tic = datetime.datetime.now()
    
    
    ## Print time scraping started
    print(f'Scraping started at: {tic}')
    
    
    ## Scrape information for each player
    for season in seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league_name_long in league_names_long:
            
            #### Determine league short name from the league names dictionary
            league_name_short = [v for k,v in dict_league_names.items() if k == league_name_long][0]
            
            #### Save Player URL List (if not already saved)
            if not os.path.exists(os.path.join(data_dir_fbref + f'/raw/outfield/{league_name_long}/{season}/fbref_outfield_player_stats_{league_name_long}_{season}_latest.csv')):

                ##### Scraping

                ##### Print statement
                print(f'Scraping started for player stats data for {league_name_long} league for the {season} season...')

                ##### Standard stats
                print(f'Scraping Standard stats...')
                url_std_stats = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fstats%2Fplayers%2F{season}-{league_name_long}&div=div_stats_standard'
                df_std_stats = pd.read_html(url_std_stats, header=1)[0]

                ##### Goalkeeper stats
                #print(f'Scraping Goalkeeper stats...')
                #url_keepers = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepers%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper'
                #df_keepers = pd.read_html(url_keepers, header=1)[0]

                ##### Advanced Goalkeeper stats
                #print(f'Scraping Advanced Goalkeeper stats...')
                #url_keepers_adv = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepersadv%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper_adv'
                #df_keepers_adv = pd.read_html(url_keepers_adv, header=1)[0]

                ##### Shooting stats
                print(f'Scraping Shooting stats...')
                url_shooting = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fshooting%2Fplayers%2F{season}-{league_name_long}&div=div_stats_shooting'
                df_shooting = pd.read_html(url_shooting, header=1)[0]

                ##### Passing stats
                print(f'Scraping Passing stats...')
                url_passing = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpassing%2Fplayers%2F{season}-{league_name_long}&div=div_stats_passing'
                df_passing = pd.read_html(url_passing, header=1)[0]

                ##### Pass Types stats
                print(f'Scraping Pass Types stats...')
                url_passing_types = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpassing_types%2Fplayers%2F{season}-{league_name_long}&div=div_stats_passing_types'
                df_passing_types = pd.read_html(url_passing_types, header=1)[0]

                ##### Goals and Shot Creation stats
                print(f'Scraping Goals and Shot Creation stats...')
                url_gca = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fgca%2Fplayers%2F{season}-{league_name_long}&div=div_stats_gca'
                df_gca = pd.read_html(url_gca, header=1)[0]

                ##### Defensive Actions stats
                print(f'Scraping Defensive Actions stats...')
                url_defense = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fdefense%2Fplayers%2F{season}-{league_name_long}&div=div_stats_defense'
                df_defense = pd.read_html(url_defense, header=1)[0]

                ##### Possession stats
                print(f'Scraping Possession stats...')
                url_possession = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fpossession%2Fplayers%2F{season}-{league_name_long}&div=div_stats_possession'
                df_possession = pd.read_html(url_possession, header=1)[0]

                ##### Playing Time stats
                print(f'Scraping Playing Time stats...')
                url_playing_time = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fplayingtime%2Fplayers%2F{season}-{league_name_long}&div=div_stats_playing_time'
                df_playing_time = pd.read_html(url_playing_time, header=1)[0]

                ##### Miscellaneous stats
                print(f'Scraping Miscellaneous stats...')
                url_misc = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fmisc%2Fplayers%2F{season}-{league_name_long}&div=div_stats_misc'
                df_misc = pd.read_html(url_misc, header=1)[0]

                ##### Concatenate defined individual DataFrames
                
                ####### Define DataFrames to be concatenated side-by-side (not all of them)
                lst_dfs = [df_std_stats, df_shooting, df_passing, df_passing_types, df_gca, df_defense, df_possession]

                ###### Concatenate DataFrames side-by-side (indicated in list above)
                df_all = pd.concat(lst_dfs, axis=1)

                ###### Drop duplicate columns
                df_all = df_all.loc[:,~df_all.columns.duplicated()]

                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()
                
                ##### Left join defined individual DataFrames
                
                ####### Define join conditions
                conditions_join = ['Player', 'Nation', 'Pos', 'Squad', 'Comp']

                ###### Left join Playing Time data
                df_all = pd.merge(df_all, df_playing_time, left_on=conditions_join, right_on=conditions_join, how='left')

                ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
                df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
                df_all.columns = df_all.columns.str.replace('_x','')
                
                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()

                ###### Left join Misc data
                df_all = pd.merge(df_all, df_misc, left_on=conditions_join, right_on=conditions_join, how='left')

                ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
                df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
                df_all.columns = df_all.columns.str.replace('_x','')
                
                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()
                
                
                ##### Engineer DataFrames
                
                ###### Take first two digits of age - fixes current season issue with extra values
                df_all['Age'] = df_all['Age'].astype(str).str[:2]
                
                ###### Create columns for league code and season
                df_all['League Name'] = league_name_long
                df_all['League ID'] = league_name_short
                df_all['Season'] = season              

                ###### Drop duplicates
                df_all = df_all.drop_duplicates()

                
                ##### Save DataFrame
                df_all.to_csv(data_dir_fbref + f'/raw/outfield/{league_name_long}/{season}/fbref_outfield_player_stats_{league_name_long}_{season}_latest.csv', index=None, header=True, encoding='utf-8')        
                
                ##### Export a copy to the 'archive' subfolder, including the date
                df_all.to_csv(data_dir_fbref + f'/raw/outfield/{league_name_long}/{season}/archive/fbref_outfield_player_stats_{league_name_long}_{season}_last_updated_{today}.csv', index=None, header=True, encoding='utf-8')        
                
                
                ##### Print statement for league and season
                print(f'All player stats data for the {league_name_long} league for {season} season scraped and saved.')
             
            
            #### Load player stats data (if already saved)
            else:

                ##### Print statement
                print(f'Player stats data for the {league_name_long} league for the {season} season already saved as a CSV file.')         

                
    ## End timer
    toc = datetime.datetime.now()
    
    
    ## Print time scraping ended
    print(f'Scraping ended at: {toc}')

    
    ## Calculate time take
    total_time = (toc-tic).total_seconds()
    print(f'Time taken to scrape the player stats data for {len(league_names_long)} leagues for {len(seasons)} seasons is: {total_time/60:0.2f} minutes.')

    
    ## Unify individual CSV files as a single DataFrame
    
    ### Show files in directory
    all_files = glob.glob(os.path.join(data_dir_fbref + f'/raw/outfield/*/*/fbref_outfield_player_stats_*_*_latest.csv'))
    
    ### Create an empty list of Players URLs
    lst_player_stats_all = []

    ### Loop through list of files and read into temporary DataFrames
    for filename in all_files:
        df_temp = pd.read_csv(filename, index_col=None, header=0)
        lst_player_stats_all.append(df_temp)

    ### Concatenate the files into a single DataFrame
    df_fbref_player_stats_all = pd.concat(lst_player_stats_all, axis=0, ignore_index=True)
    
    ### Drop header row of each concatenated  DataFrame (contains 'Rk', 'Rk' column)
    df_fbref_player_stats_all = df_fbref_player_stats_all[~df_fbref_player_stats_all['Rk'].str.contains('Rk')]
    
    ### Drop 'Rk' column
    df_fbref_player_stats_all = df_fbref_player_stats_all.drop(['Rk'], axis=1)
    
    ### Reset index
    #df_fbref_player_stats_all = df_fbref_player_stats_all.reset_index()
    
    ### Sort DataFrame
    df_fbref_player_stats_all = df_fbref_player_stats_all.sort_values(['League Name', 'Season', 'Player'], ascending=[True, True, True])

    
    ## Export DataFrame
    
    ###
    df_fbref_player_stats_all.to_csv(data_dir_fbref + f'/raw/outfield/fbref_outfield_player_stats_combined_latest.csv', index=None, header=True, encoding='utf-8')
    
    ### Save a copy to archive folder (dated)
    df_fbref_player_stats_all.to_csv(data_dir_fbref + f'/raw/outfield/archive/fbref_outfield_player_stats_combined_last_updated_{today}.csv', index=None, header=True, encoding='utf-8')
    
    
    ## Distinct number of players
    total_players = df_fbref_player_stats_all['Player'].nunique()


    ## Print statement
    print(f'Player stats DataFrame contains {total_players} players.')
    
    
    ## Return final list of Player URLs
    return(df_fbref_player_stats_all)

In [19]:
# Define function for scraping a defined season and competition of FBref player data
def get_fbref_goalkeeper_stats(lst_league_names, lst_seasons):
    
    """
    Function to scrape goalkeeper stats from FBref.
    """
    
    
    ## Define list of league names
    league_names_long = lst_league_names
    
    
    ## Define seasons to scrape
    seasons = lst_seasons
    
    
    ## Start timer
    tic = datetime.datetime.now()
    
    
    ## Print time scraping started
    print(f'Scraping started at: {tic}')
    
    
    ## Scrape information for each player
    for season in seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league_name_long in league_names_long:
            
            #### Determine league short name from the league names dictionary
            league_name_short = [v for k,v in dict_league_names.items() if k == league_name_long][0]
            
            #### Save Player URL List (if not already saved)
            if not os.path.exists(os.path.join(data_dir_fbref + f'/raw/goalkeeper/{league_name_long}/{season}/fbref_goalkeeper_stats_{league_name_long}_{season}_latest.csv', encoding='utf-8')):

                ##### Scraping

                ##### Print statement
                print(f'Scraping started for goalkeeper stats data for {league_name_long} league for the {season} season...')

                ##### Standard stats
                print(f'Scraping Standard stats...')
                url_std_stats = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fstats%2Fplayers%2F{season}-{league_name_long}&div=div_stats_standard'
                df_std_stats = pd.read_html(url_std_stats, header=1)[0]

                ##### Goalkeeper stats
                print(f'Scraping Goalkeeper stats...')
                url_keepers = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepers%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper'
                df_keepers = pd.read_html(url_keepers, header=1)[0]

                ##### Advanced Goalkeeper stats
                print(f'Scraping Advanced Goalkeeper stats...')
                url_keepers_adv = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fkeepersadv%2Fplayers%2F{season}-{league_name_long}&div=div_stats_keeper_adv'
                df_keepers_adv = pd.read_html(url_keepers_adv, header=1)[0]

                ##### Playing Time stats
                print(f'Scraping Playing Time stats...')
                url_playing_time = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fplayingtime%2Fplayers%2F{season}-{league_name_long}&div=div_stats_playing_time'
                df_playing_time = pd.read_html(url_playing_time, header=1)[0]

                ##### Miscellaneous stats
                print(f'Scraping Miscellaneous stats...')
                url_misc = f'https://widgets.sports-reference.com/wg.fcgi?css=1&site=fb&url=%2Fen%2Fcomps%2F{league_name_short}%2F{season}%2Fmisc%2Fplayers%2F{season}-{league_name_long}&div=div_stats_misc'
                df_misc = pd.read_html(url_misc, header=1)[0]

                ##### Concatenate defined individual DataFrames
                
                ####### Define DataFrames to be concatenated side-by-side (not all of them)
                lst_dfs = [df_keepers, df_keepers_adv]

                ###### Concatenate DataFrames side-by-side (indicated in list above)
                df_all = pd.concat(lst_dfs, axis=1)

                ###### Drop duplicate columns
                df_all = df_all.loc[:,~df_all.columns.duplicated()]

                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()
                
                ##### Left join defined individual DataFrames
                
                ####### Define join conditions
                conditions_join = ['Player', 'Nation', 'Pos', 'Squad', 'Comp']

                ###### Left join Standard Stats data
                df_all = pd.merge(df_all, df_std_stats, left_on=conditions_join, right_on=conditions_join, how='left')

                ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
                df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
                df_all.columns = df_all.columns.str.replace('_x','')
                
                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()
                
                ###### Left join Playing Time data
                df_all = pd.merge(df_all, df_playing_time, left_on=conditions_join, right_on=conditions_join, how='left')

                ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
                df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
                df_all.columns = df_all.columns.str.replace('_x','')
                
                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()

                ###### Left join Misc data
                df_all = pd.merge(df_all, df_misc, left_on=conditions_join, right_on=conditions_join, how='left')

                ###### Remove duplicate columns after join (contain '_y') and remove '_x' suffix from kept columns
                df_all = df_all[df_all.columns.drop(list(df_all.filter(regex='_y')))]
                df_all.columns = df_all.columns.str.replace('_x','')
                
                ###### Drop duplicate rows
                df_all = df_all.drop_duplicates()
                
                
                ##### Engineer DataFrames
                
                ###### Take first two digits of age - fixes current season issue with extra values
                df_all['Age'] = df_all['Age'].astype(str).str[:2]
                
                ###### Create columns for league code and season
                df_all['League Name'] = league_name_long
                df_all['League ID'] = league_name_short
                df_all['Season'] = season              

                ###### Drop duplicates
                df_all = df_all.drop_duplicates()

                
                ##### Save DataFrame
                df_all.to_csv(data_dir_fbref + f'/raw/goalkeeper/{league_name_long}/{season}/fbref_goalkeeper_stats_{league_name_long}_{season}_latest.csv', index=None, header=True, encoding='utf-8')        
                
                ##### Export a copy to the 'archive' subfolder, including the date
                df_all.to_csv(data_dir_fbref + f'/raw/goalkeeper/{league_name_long}/{season}/archive/fbref_goalkeeper_stats_{league_name_long}_{season}_last_updated_{today}.csv', index=None, header=True, encoding='utf-8')        
                
                
                ##### Print statement for league and season
                print(f'All Goalkeeper stats data for the {league_name_long} league for {season} season scraped and saved.')
             
            
            #### Load goalkeeper stats data (if already saved)
            else:

                ##### Print statement
                print(f'Goalkeeper stats data for the {league_name_long} league for the {season} season already saved as a CSV file.')         

                
    ## End timer
    toc = datetime.datetime.now()
    
    
    ## Print time scraping ended
    print(f'Scraping ended at: {toc}')

    
    ## Calculate time take
    total_time = (toc-tic).total_seconds()
    print(f'Time taken to scrape the goalkeeper stats data for {len(league_names_long)} leagues for {len(seasons)} seasons is: {total_time/60:0.2f} minutes.')

    
    ## Unify individual CSV files as a single DataFrame
    
    ### Show files in directory
    all_files = glob.glob(os.path.join(data_dir_fbref + f'/raw/goalkeeper/*/*/fbref_goalkeeper_stats_*_*_latest.csv'))
    
    ### Create an empty list of Players URLs
    lst_goalkeeper_stats_all = []

    ### Loop through list of files and read into temporary DataFrames
    for filename in all_files:
        df_temp = pd.read_csv(filename, index_col=None, header=0)
        lst_goalkeeper_stats_all.append(df_temp)

    ### Concatenate the files into a single DataFrame
    df_fbref_goalkeeper_stats_all = pd.concat(lst_goalkeeper_stats_all, axis=0, ignore_index=True)
    
    ### Drop header row of each concatenated  DataFrame (contains 'Rk', 'Rk' column)
    df_fbref_goalkeeper_stats_all = df_fbref_goalkeeper_stats_all[~df_fbref_goalkeeper_stats_all['Rk'].str.contains('Rk')]
    
    ### Drop 'Rk' column
    df_fbref_goalkeeper_stats_all = df_fbref_goalkeeper_stats_all.drop(['Rk'], axis=1)
    
    ### Reset index
    #df_fbref_goalkeeper_stats_all = df_fbref_goalkeeper_stats_all.reset_index()
    
    ### Sort DataFrame
    df_fbref_goalkeeper_stats_all = df_fbref_goalkeeper_stats_all.sort_values(['League Name', 'Season', 'Player'], ascending=[True, True, True])

    
    ## Export DataFrame
    
    ###
    df_fbref_goalkeeper_stats_all.to_csv(data_dir_fbref + f'/raw/goalkeeper/fbref_goalkeeper_stats_combined_latest.csv', index=None, header=True, encoding='utf-8')
    
    ### Save a copy to archive folder (dated)
    df_fbref_goalkeeper_stats_all.to_csv(data_dir_fbref + f'/raw/goalkeeper/archive/fbref_goalkeeper_stats_combined_last_updated_{today}.csv', index=None, header=True, encoding='utf-8')
    
    
    ## Distinct number of goalkeepers
    total_players = df_fbref_goalkeeper_stats_all['Player'].nunique()


    ## Print statement
    print(f'Goalkeeper stats DataFrame contains {total_players} players.')
    
    
    ## Return final list of Player URLs
    return(df_fbref_goalkeeper_stats_all)

### Create Directory Structure
Create folders and subfolders for data, if not already created.

In [20]:
# Make the data directory structure
for folder in lst_folders:
    path = os.path.join(data_dir_fbref, folder)
    if not os.path.exists(path):
        os.mkdir(path)
        for data_types in lst_data_types:
            path = os.path.join(data_dir_fbref, folder, data_types)
            if not os.path.exists(path):
                os.mkdir(path)
                os.mkdir(os.path.join(path, 'archive'))
                for league in lst_league_names_long:
                    path = os.path.join(data_dir_fbref, folder, data_types, league)
                    if not os.path.exists(path):
                        os.mkdir(path)
                        for season in lst_seasons:
                            path = os.path.join(data_dir_fbref, folder, data_types, league, season)
                            if not os.path.exists(path):
                                os.mkdir(path)
                                os.mkdir(os.path.join(path, 'archive'))

### Notebook Settings

In [21]:
# Display all columns of pandas DataFrames
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
This Jupyter notebook is part of a series of notebooks, to scrape, parse, engineer, and unify datasets, that can be used for modeling purposes.

This particular notebook is one of several web scraping notebooks, that takes data from [FBref](https://fbref.com/en/), provided by [StatsBomb](https://statsbomb.com/), and scrapes it using the [pandas](http://pandas.pydata.org/) [`read_html`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html) function and manipulates it as Dataframe.

This notebook, along with the other notebooks in this project workflow are shown in the following diagram:

![roadmap](../../img/football_analytics_data_roadmap.png)

Links to these notebooks in the [`football_analytics`](https://github.com/eddwebster/football_analytics) GitHub repository can be found at the following:
*    [1. Webscraping](https://github.com/eddwebster/football_analytics/tree/master/notebooks/1_data_scraping)
     +    [FBref Player Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Player%20Stats%20Web%20Scraping.ipynb)
     +    [TransferMarket Player Bio and Status Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Bio%20and%20Status%20Web%20Scraping.ipynb)
     +    [TransferMarket Player Valuation Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Valuation%20Web%20Scraping.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Webscraping.ipynb)
     +    [Capology Player Salary Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/Capology%20Player%20Salary%20Web%20Scraping.ipynb)
     +    [FBref Team Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Team%20Stats%20Web%20Scraping.ipynb)
*    [2. Data Parsing](https://github.com/eddwebster/football_analytics/tree/master/notebooks/2_data_parsing)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/2_data_parsing/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
*    [3. Data Engineering](https://github.com/eddwebster/football_analytics/tree/master/notebooks/3_data_engineering)
     +    [FBref Player Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Player%20Stats%20Data%20Engineering.ipynb)
     +    [TransferMarket Player Bio and Status Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Bio%20and%20Status%20Data%20Engineering.ipynb)
     +    [TransferMarket Player Valuation Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Valuation%20Data%20Engineering.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb)
     +    [Capology Player Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb)
     +    [FBref Team Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Team%20Stats%20Data%20Engineering.ipynb)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
     +    [TransferMarkt Team Recorded Transfer Fee Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Team%20Recorded%20Transfer%20Fee%20Data%20Engineering.ipynb) (aggregated from [TransferMarkt Player Recorded Transfer Fees notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb))
     +    [Capology Team Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Team%20Salary%20Data%20Engineering.ipynb) (aggregated from [Capology Player Salary notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb))
*    [4. Data Unification](https://github.com/eddwebster/football_analytics/tree/master/notebooks/4_data_unification)
*    [5. Modeling and Data Analysis]()

---

<a id='section3'></a>

## <a id='#section3'>3. Data Scraping</a>

<a id='section3.1'></a>

### <a id='#section3.1'>3.1. Introduction</a>
This Data Sources section has been has been split into two subsections - outfielder, and goalkeeper.

The data needs to be scraped, converted to a pandas DataFrame ([Section 3](#section3)) and cleaned in the Data Engineering section ([Section 4](#section4)).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

From FBref it is also possible to scrape team data, this is covered separately in the following notebook in the [Data Scraping](https://github.com/eddwebster/football_analytics/tree/master/notebooks/1_data_scraping) folder [[link](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Team%20Stats%20Web%20Scraping.ipynb)].

<a id='section3.2'></a>

### <a id='#section3.2'>3.2. Outfield Players</a>

<a id='section3.2.1'></a>

#### <a id='#section3.2.1'>3.2.1. Data Dictionary</a>
The raw dataset has one hundred and sixty four features (columns) with the following definitions and data types:

| Variable     | Data Type    | Description    |
|------|-----|-----|
| `squad`    | object    | Squad name e.g. Arsenal    |
| `players_used`    | float64    | Number of Players used in Games    |
| `possession`    | float64    | Percentage of time with possession of the ball    |

[MORE DEFINITIONS TO BE ADDED]

<a id='section3.2.2'></a>

#### <a id='#section3.2.2'>3.2.2. Creating the DataFrame - scraping the data</a>
The data is scraped and saved as a DataFrame using the pandas `read_html()` function [[link](https://stackoverflow.com/questions/66517625/attributeerror-nonetype-object-has-no-attribute-text-beautifulshop)].

In [22]:
lst_league_names = ['Big-5-European-Leagues']     #'Premier-League', 'Ligue-1', 'Bundesliga', 'Serie-A', 'La-Liga', 'Major-League-Soccer']
lst_seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

df_fbref_outfield_raw = get_fbref_player_stats(lst_league_names, lst_seasons)

Scraping started at: 2021-11-03 13:21:11.712525
Scraping started for the 2017-2018 season...
Scraping started for player stats data for Big-5-European-Leagues league for the 2017-2018 season...
Scraping Standard stats...
Scraping Shooting stats...
Scraping Passing stats...
Scraping Pass Types stats...
Scraping Goals and Shot Creation stats...
Scraping Defensive Actions stats...
Scraping Possession stats...
Scraping Playing Time stats...
Scraping Miscellaneous stats...
All player stats data for the Big-5-European-Leagues league for 2017-2018 season scraped and saved.
Scraping started for the 2018-2019 season...
Scraping started for player stats data for Big-5-European-Leagues league for the 2018-2019 season...
Scraping Standard stats...
Scraping Shooting stats...
Scraping Passing stats...
Scraping Pass Types stats...
Scraping Goals and Shot Creation stats...
Scraping Defensive Actions stats...
Scraping Possession stats...
Scraping Playing Time stats...
Scraping Miscellaneous stats...
Al

HTTPError: HTTP Error 504: Gateway Time-out

<a id='section3.2.3'></a>

#### <a id='#section3.2.3'>3.2.3. Preliminary Data Handling</a>

<a id='section3.2.3.1'></a>

##### <a id='#section3.2.3.1'>3.2.3.1. Summary Report</a>
Initial step of the data handling and Exploratory Data Analysis (EDA) is to create a quick summary report of the dataset using [pandas Profiling Report](https://github.com/pandas-profiling/pandas-profiling).

In [ ]:
# Summary of the data using pandas Profiling Report
pp.ProfileReport(df_fbref_outfield_raw)

<a id='section3.2.3.2'></a>

##### <a id='#section3.2.3.2'>3.2.3.2. Further Inspection</a>
The following commands go into more bespoke summary of the dataset. Some of the commands include content covered in the [pandas Profiling](https://github.com/pandas-profiling/pandas-profiling) summary above, but using the standard [pandas](https://pandas.pydata.org/) functions and methods that most peoplem will be more familiar with.

First check the quality of the dataset by looking first and last rows in pandas using the [head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [tail()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html) methods.

In [ ]:
# Display the first five rows of the raw DataFrame, df_fbref_outfield_raw
df_fbref_outfield_raw.head()

In [ ]:
# Display the last five rows of the raw DataFrame, df_fbref_outfield_raw
df_fbref_outfield_raw.tail()

[shape](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) returns a tuple representing the dimensionality of the DataFrame.

In [ ]:
# Print the shape of the raw DataFrame, df_fbref_outfield_raw
print(df_fbref_outfield_raw.shape)

[columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html) returns the column labels of the DataFrame.

In [ ]:
# Features (column names) of the raw DataFrame, df_fbref_outfield_raw
df_fbref_outfield_raw.columns

The [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) method returns the data types of each attribute in the DataFrame.

In [ ]:
# Displays all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_fbref_outfield_raw.dtypes)

The [info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) method to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.

In [ ]:
# Info for the raw DataFrame, df_fbref_outfield_raw
df_fbref_outfield_raw.info()

The [describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) method to show some useful statistics for each numerical column in the DataFrame.

In [ ]:
# Description of the raw DataFrame, df_fbref_outfield_raw, showing some summary statistics for each numberical column in the DataFrame
df_fbref_outfield_raw.describe()

Next, we will check to see how many missing values we have i.e. the number of NULL values in the dataset, and in what features these missing values are located. This can be plotted nicely using the [missingno](https://pypi.org/project/missingno/) library (pip install missingno).

In [ ]:
# Plot visualisation of the missing values for each feature of the raw DataFrame, df_fbref_outfield_raw
msno.matrix(df_fbref_outfield_raw, figsize = (30, 7))

In [ ]:
# Counts of missing values
null_value_stats = df_fbref_outfield_raw.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

The visualisation shows us very quickly that there are missing values in the dataset but as this data is scraped, this fine at this stage.

### <a id='#section3.4'>3.4. Goalkeepers</a>

#### <a id='#section3.4.1'>3.4.1. Data Dictionary</a>
The raw dataset has one hundred and eighty eight features (columns) with the following definitions and data types:

| Variable     | Data Type    | Description    |
|------|-----|-----|
| `squad`    | object    | Squad name e.g. Arsenal    |
| `players_used`    | float64    | Number of Players used in Games    |
| `possession`    | float64    | Percentage of time with possession of the ball    |

[MORE DEFINITIONS TO BE ADDED]

#### <a id='#section3.4.2'>3.4.2. Creating the DataFrame - scraping the data</a>
Scrape the data and save as a pandas DataFrame using the function `get_keeper_data`.

Like the outfielders, to download the goalkeeper data we are not required to download the data for individual leagues and concatenate them, they can be downloaded as one from the 'Big 5' European leagues goalkeepers page.

In [ ]:
lst_league_names = ['Big-5-European-Leagues']     #'Premier-League', 'Ligue-1', 'Bundesliga', 'Serie-A', 'La-Liga', 'Major-League-Soccer']
lst_seasons = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022']

df_fbref_goalkeeper_raw = get_fbref_goalkeeper_stats(lst_league_names, lst_seasons)

#### <a id='#section3.4.3'>3.4.3. Preliminary Data Handling</a>
Let's quality of the dataset by looking first and last rows in pandas using the [head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [tail()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.tail.html) methods.

In [ ]:
# Display the first five rows of the raw DataFrame, df_fbref_goalkeeper_raw
df_fbref_goalkeeper_raw.head()

In [ ]:
# Display the last five rows of the raw DataFrame, df_fbref_goalkeeper_raw
df_fbref_goalkeeper_raw.tail()

[shape](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) returns a tuple representing the dimensionality of the DataFrame.

In [ ]:
# Print the shape of the raw DataFrame, df_fbref_goalkeeper_raw
print(df_fbref_goalkeeper_raw.shape)

[columns](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.columns.html) returns the column labels of the DataFrame.

In [ ]:
# Features (column names) of the raw DataFrame, df_fbref_goalkeeper_raw
df_fbref_goalkeeper_raw.columns

The [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) method returns the data types of each attribute in the DataFrame.

In [ ]:
# Displays all columns
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_fbref_goalkeeper_raw.dtypes)

The [info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) method to get a quick description of the data, in particular the total number of rows, and each attribute’s type and number of non-null values.

In [ ]:
# Info for the raw DataFrame, df_fbref_goalkeeper_raw
df_fbref_goalkeeper_raw.info()

The [describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) method to show some useful statistics for each numerical column in the DataFrame.

In [ ]:
# Description of the raw DataFrame, df_fbref_goalkeeper_raw, showing some summary statistics for each numberical column in the DataFrame
df_fbref_goalkeeper_raw.describe()

Next, we will check to see how many missing values we have i.e. the number of NULL values in the dataset, and in what features these missing values are located. This can be plotted nicely using the [missingno](https://pypi.org/project/missingno/) library (pip install missingno).

In [ ]:
# Plot visualisation of the missing values for each feature of the raw DataFrame, df_fbref_goalkeeper_raw
msno.matrix(df_fbref_goalkeeper_raw, figsize = (30, 7))

In [ ]:
# Counts of missing values
null_value_stats = df_fbref_goalkeeper_raw.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

The visualisation shows us very quickly that there are missing values in the dataset but as this data is scraped, this fine at this stage.

___

<a id='section4'></a>

## <a id='#section4'>4. Summary</a>
This notebook scrapes player statstics data from [FBref](https://fbref.com/en/) via [StatsBomb](https://statsbomb.com/), using [pandas](http://pandas.pydata.org/) for data manipulation through DataFrames.

With this notebook we now have aggregated player performance data for players in the 'Big 5' European leagues for the 17/18-present seasons.

___

<a id='section5'></a>

## <a id='#section5'>5. Next Steps</a>
This data is now ready to be engineered before being matched to other datasets such as data from [TransferMarkt](https://www.transfermarkt.co.uk/) and [Capology](https://www.capology.com/).

The Data Engineering subfolder in GitHub can be found [here](https://github.com/eddwebster/football_analytics/tree/master/notebooks/B\)%20Data%20Engineering) and a static version of the FBref data engineering notebookecord can be found [here](https://nbviewer.org/github/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Player%20Stats%20Data%20Engineering.ipynb).

___

<a id='section6'></a>

## <a id='#section6'>6. References</a>

#### Data and Web Scraping
*    [FBref](https://fbref.com/) for the data to scrape
*    FBref statement for using StatsBomb's data: https://fbref.com/en/statsbomb/
*    [StatsBomb](https://statsbomb.com/) providing the data to FBref
*    [FBref_EPL GitHub repository](https://github.com/chmartin/FBref_EPL) by [chmartin](https://github.com/chmartin) for the original web scraping code
*    [Scrape-FBref-data GitHub repository](https://github.com/parth1902/Scrape-FBref-data) by [parth1902](https://github.com/parth1902) for the revised web scraping code for the new FBref metrics


#### Countries
*    [Comparison of alphabetic country codes Wiki](https://en.wikipedia.org/wiki/Comparison_of_alphabetic_country_codes)

---

***Visit my website [eddwebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)